## Sixth Session (Related to the Course Project)

---------------

## Graph Classification with [Deep Graph Library (DGL)](https://docs.dgl.ai/index.html) for the graduate course "[Graph Machine learning](https://github.com/zahta/graph_ml)"

### Dataset: BBBP

##### by [Zahra Taheri](https://github.com/zahta), 06 June 2023

---------------

### This Tutorial Is Prepared Based on the Following References

- [FunQG: Molecular Representation Learning via Quotient Graphs](https://pubs.acs.org/doi/10.1021/acs.jcim.3c00445)
- [Supporting Information of FunQG](https://pubs.acs.org/doi/suppl/10.1021/acs.jcim.3c00445/suppl_file/ci3c00445_si_001.pdf)
- [GitHub Repository of FunQG](https://github.com/hhaji/funqg)

In [ ]:
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.5 MB/s eta 0:00:00


In [ ]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv

#### Set Path

In [ ]:
current_dir = "./"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/"

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True)

path_save = current_dir + "graph_data.zip"
shutil.unpack_archive(path_save, folder_data_temp)

#### Custom PyTorch Datasets

In [ ]:
""" Classification Dataset """
class DGLDatasetClass(torch.utils.data.Dataset):
    def __init__(self, address):
        # Constructor method for the dataset
        # address: the file path of the dataset
        self.address=address+".bin"
        # Load the dataset from the given file path
        self.list_graphs, train_labels_masks_globals = dgl.load_graphs(self.address)
        num_graphs =len(self.list_graphs)
        # Extract the labels, masks and global features from the dataset
        self.labels = train_labels_masks_globals["labels"].view(num_graphs,-1)
        self.masks = train_labels_masks_globals["masks"].view(num_graphs,-1)
        self.globals = train_labels_masks_globals["globals"].view(num_graphs,-1)

    def __len__(self):
        # Method to return the length of the dataset
        return len(self.list_graphs)

    def __getitem__(self, idx):
        # Method to return a single sample from the dataset
        # idx: the index of the sample to be returned
        return  self.list_graphs[idx], self.labels[idx], self.masks[idx], self.globals[idx]

#### Defining Train, Validation, and Test Set

In [ ]:
path_data_temp = folder_data_temp + "scaffold"+"_"+str(0)
train_set = DGLDatasetClass(address=path_data_temp+"_train")
val_set = DGLDatasetClass(address=path_data_temp+"_val")
test_set = DGLDatasetClass(address=path_data_temp+"_test")

print(len(train_set), len(val_set), len(test_set))


1631 203 205


#### Data Loader

In [ ]:
def collate(batch):
    # batch is a list of tuples (graphs, labels, masks, globals)
    # Concatenate a sequence of graphs
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    # Concatenate a sequence of tensors (labels) along a new dimension
    labels = [e[1] for e in batch]
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (masks) along a new dimension
    masks = [e[2] for e in batch]
    masks = torch.stack(masks, 0)

    # Concatenate a sequence of tensors (globals) along a new dimension
    globals = [e[3] for e in batch]
    globals = torch.stack(globals, 0)

    return g, labels, masks, globals


def loader(batch_size):
    # This function returns a set of data loaders for the training, validation, and test sets
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=False,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [ ]:
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=32)

#### Defining A GNN

##### Some Variables

In [ ]:
#BBBP dataset has 1 task. Some other datasets may have some more number of tasks, e.g., tox21 has 12 tasks.
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}


#GNN 1

In [ ]:
class GNN1(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        # Create two GraphConv layers for the message passing
        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size,allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks,allow_zero_in_degree=True)

    # Define the forward method of the model
    def forward(self, mol_dgl_graph, globals):
        # The forward method takes a molecular graph and a global feature vector as input
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        # Truncate the node feature size to the specified size
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        # Truncate the edge feature size to the specified size
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        # Store the final node representations as node features in the graph
        return dgl.mean_nodes(mol_dgl_graph, "h")
        # Compute the mean of the node features across all nodes and return it as the output of the model

#### Function to Compute Score of the Model

In [ ]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    # A function to compute the performance score of the model on the given data
    # model: the trained model to be evaluated
    # data_loader: the data loader for the validation or test set
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        # Initialize empty tensors to store the predictions, labels, and masks
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        # Iterate over the data loader and make predictions for each batch
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            # Concatenate the predictions, labels, and masks for all batches
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        # Compute the average score across all tasks
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                # Compute the metric score for the current task
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                # If the metric cannot be computed, set the score to 0
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

In [ ]:
def loss_func(output, label, mask, num_tasks):
    # A function to compute the loss between the model's output and the true labels
    # mask: the binary mask indicating which samples should be included in the loss calculation
    pos_weight = torch.ones((1, num_tasks))
    # Create a tensor of ones with the same shape as the output tensor (to be used as positive weights)
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    # Create a loss function that applies binary cross-entropy with logits
    # The 'none' reduction parameter is used to keep the loss for each sample separate
    # The positive weights are used to give more weight to the positive class (to handle class imbalance)
    loss = mask*criterion(output,label)
    # Compute the loss for each sample, but only include the samples specified by the mask
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

In [ ]:
def train_epoch(train_dataloader, model, optimizer):
    # A function to train the model for one epoch on the given data
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        # Iterate over the data loader and get the input samples
        prediction = model(mol_dgl_graph, globals)
        # Make predictions for the input samples using the model
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        # Compute the loss between the predictions and the true labels
        optimizer.zero_grad(set_to_none=True)
        # Zero the gradients of the optimizer (to prevent accumulation from previous iterations)
        loss_train.backward()
        # Compute the gradients of the loss with respect to the model parameters
        optimizer.step()
        # Update the model parameters using the computed gradients
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [ ]:
def train_evaluate():
    # A function to train and evaluate the model
    model = GNN1(config, global_size, num_tasks)
    # Create a new instance of the GNN1 model
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
    # Create an Adam optimizer to update the model parameters during training
    best_val = 0
    patience_count = 1
    epoch = 1
    while epoch <= num_epochs:
        # Train and evaluate the model for the specified number of epochs
        if patience_count <= patience:
            # Check if the patience count has been exceeded
            model.train()
            # Set the model to training mode
            loss_train = train_epoch(train_dataloader, model, optimizer)
            # Train the model on the training set for one epoch
            model.eval()
            # Set the model to evaluation mode
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            # Evaluate the model on the validation set and compute the validation score
            if score_val > best_val:
                # Check if the current validation score is better than the best validation score so far
                best_val = score_val
                # If so, update the best validation score and save a checkpoint of the model
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                # If not, increment the patience count
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))
            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # Save the best model
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)
    # Remove any existing best model and copy the checkpoint to the best model path
    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")
    # Print the final average validation score

##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    # A function to evaluate the trained model on the test set
    final_model = GNN1(config, global_size, num_tasks)
    # Create a new instance of the GNN1 model
    path = os.path.join(best_model_path, 'checkpoint.pth')
    # Load the best model checkpoint from the best model path
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    # Load the saved model state dictionary into the final model
    final_model.eval()
    # Set the final model to evaluation mode
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)
    # Evaluate the final model on the test set and compute the test score
    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.608 | Valid Score: 0.450
 
Epoch: 1/100 | Best Valid Score Until Now: 0.450 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.541 | Valid Score: 0.738
 
Epoch: 2/100 | Best Valid Score Until Now: 0.738 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.503 | Valid Score: 0.807
 
Epoch: 3/100 | Best Valid Score Until Now: 0.807 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.474 | Valid Score: 0.821
 
Epoch: 4/100 | Best Valid Score Until Now: 0.821 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.458 | Valid Score: 0.823
 
Epoch: 5/100 | Best Valid Score Until Now: 0.823 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.443 | Valid Score: 0.824
 
Epoch: 6/100 | Best Valid Score Until Now: 0.824 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.436 | Valid Score: 0.829
 
Epoch: 7/100 | Best Valid Score Until Now: 0.829 

Patience 1
Epoch: 8/100 | Training Loss: 0.430 | Valid Score: 0.826
 
Epoch: 8/100 | Best Valid Score Until Now: 0.829

#GNN 2

In [ ]:
class GNN2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size,allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size,allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.num_tasks,allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

#### Function to Compute Score of the Model

In [ ]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

In [ ]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

In [ ]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [ ]:
def train_evaluate():

    model = GNN2(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GNN2(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.572 | Valid Score: 0.798
 
Epoch: 1/100 | Best Valid Score Until Now: 0.798 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.495 | Valid Score: 0.843
 
Epoch: 2/100 | Best Valid Score Until Now: 0.843 

Patience 1
Epoch: 3/100 | Training Loss: 0.448 | Valid Score: 0.832
 
Epoch: 3/100 | Best Valid Score Until Now: 0.843 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.441 | Valid Score: 0.845
 
Epoch: 4/100 | Best Valid Score Until Now: 0.845 

Patience 1
Epoch: 5/100 | Training Loss: 0.428 | Valid Score: 0.841
 
Epoch: 5/100 | Best Valid Score Until Now: 0.845 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.413 | Valid Score: 0.848
 
Epoch: 6/100 | Best Valid Score Until Now: 0.848 

Patience 1
Epoch: 7/100 | Training Loss: 0.412 | Valid Score: 0.841
 
Epoch: 7/100 | Best Valid Score Until Now: 0.848 

Patience 2
Epoch: 8/100 | Training Loss: 0.405 | Valid Score: 0.841
 
Epoch: 8/100 | Best Valid Score Until Now: 0.848 

Patience 3
E

#GNN 3

In [ ]:
class GNN3(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)


        self.conv1 = GraphConv(self.node_feature_size, 64 ,allow_zero_in_degree=True)
        self.conv2 = GraphConv(64, 128,allow_zero_in_degree=True)
        self.conv3 = GraphConv(128, 256,allow_zero_in_degree=True)
        self.conv4 = GraphConv(256, 128,allow_zero_in_degree=True)
        self.conv5 = GraphConv(128, self.num_tasks,allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.2)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.bn1(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.bn2(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.bn3(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.bn2(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

#### Function to Compute Score of the Model

In [ ]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

In [ ]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

In [ ]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [ ]:
def train_evaluate():

    model = GNN3(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GNN3(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.532 | Valid Score: 0.836
 
Epoch: 1/100 | Best Valid Score Until Now: 0.836 

Patience 1
Epoch: 2/100 | Training Loss: 0.417 | Valid Score: 0.812
 
Epoch: 2/100 | Best Valid Score Until Now: 0.836 

Patience 2
Epoch: 3/100 | Training Loss: 0.384 | Valid Score: 0.812
 
Epoch: 3/100 | Best Valid Score Until Now: 0.836 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.351 | Valid Score: 0.839
 
Epoch: 4/100 | Best Valid Score Until Now: 0.839 

Patience 1
Epoch: 5/100 | Training Loss: 0.346 | Valid Score: 0.808
 
Epoch: 5/100 | Best Valid Score Until Now: 0.839 

Patience 2
Epoch: 6/100 | Training Loss: 0.320 | Valid Score: 0.820
 
Epoch: 6/100 | Best Valid Score Until Now: 0.839 

Patience 3
Epoch: 7/100 | Training Loss: 0.332 | Valid Score: 0.827
 
Epoch: 7/100 | Best Valid Score Until Now: 0.839 

Patience 4
Epoch: 8/100 | Training Loss: 0.311 | Valid Score: 0.799
 
Epoch: 8/100 | Best Valid Score Until Now: 0.839 

Patience 5
Epoch: 9/10

#Graph SAGE 1

In [ ]:
import dgl.function as fn
from dgl.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size, aggregator_type='mean')
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks, aggregator_type='mean')

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

#### Function to Compute Score of the Model

In [ ]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

In [ ]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

In [ ]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [ ]:
def train_evaluate():

    model = GraphSAGE(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GraphSAGE(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.586 | Valid Score: 0.780
 
Epoch: 1/100 | Best Valid Score Until Now: 0.780 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.469 | Valid Score: 0.835
 
Epoch: 2/100 | Best Valid Score Until Now: 0.835 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.437 | Valid Score: 0.842
 
Epoch: 3/100 | Best Valid Score Until Now: 0.842 

Patience 1
Epoch: 4/100 | Training Loss: 0.420 | Valid Score: 0.835
 
Epoch: 4/100 | Best Valid Score Until Now: 0.842 

Patience 2
Epoch: 5/100 | Training Loss: 0.422 | Valid Score: 0.831
 
Epoch: 5/100 | Best Valid Score Until Now: 0.842 

Patience 3
Epoch: 6/100 | Training Loss: 0.401 | Valid Score: 0.831
 
Epoch: 6/100 | Best Valid Score Until Now: 0.842 

Patience 4
Epoch: 7/100 | Training Loss: 0.394 | Valid Score: 0.836
 
Epoch: 7/100 | Best Valid Score Until Now: 0.842 

Patience 5
Epoch: 8/100 | Training Loss: 0.389 | Valid Score: 0.826
 
Epoch: 8/100 | Best Valid Score Until Now: 0.842 

Patience 6
Epoch:

#Graph SAGE 2


In [ ]:
import dgl.function as fn
from dgl.nn import SAGEConv

class GraphSAGE2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size, aggregator_type='mean')
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size , aggregator_type='mean')
        self.conv3 = SAGEConv(self.hidden_size, self.num_tasks, aggregator_type='mean')

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

#### Function to Compute Score of the Model

In [ ]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

In [ ]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

In [ ]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [ ]:
def train_evaluate():

    model = GraphSAGE2(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GraphSAGE2(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.513 | Valid Score: 0.774
 
Epoch: 1/100 | Best Valid Score Until Now: 0.774 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.422 | Valid Score: 0.802
 
Epoch: 2/100 | Best Valid Score Until Now: 0.802 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.405 | Valid Score: 0.814
 
Epoch: 3/100 | Best Valid Score Until Now: 0.814 

Patience 1
Epoch: 4/100 | Training Loss: 0.389 | Valid Score: 0.799
 
Epoch: 4/100 | Best Valid Score Until Now: 0.814 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.376 | Valid Score: 0.823
 
Epoch: 5/100 | Best Valid Score Until Now: 0.823 

Patience 1
Epoch: 6/100 | Training Loss: 0.363 | Valid Score: 0.813
 
Epoch: 6/100 | Best Valid Score Until Now: 0.823 

Patience 2
Epoch: 7/100 | Training Loss: 0.351 | Valid Score: 0.822
 
Epoch: 7/100 | Best Valid Score Until Now: 0.823 

Patience 3
Epoch: 8/100 | Training Loss: 0.359 | Valid Score: 0.817
 
Epoch: 8/100 | Best Valid Score Until Now: 0.823 

Patience 4
E

#Graph SAGE 3


In [ ]:
from dgl.nn.pytorch.conv import SAGEConv
class GraphSAGE3(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, 64 ,aggregator_type='lstm')
        self.conv2 = SAGEConv(64, 128,aggregator_type='lstm')
        self.conv3 = SAGEConv(128, 256,aggregator_type='lstm')
        self.conv4 = SAGEConv(256, 128,aggregator_type='lstm')
        self.conv5 = SAGEConv(128, self.num_tasks,aggregator_type='lstm')

        self.dropout = nn.Dropout(p=0.2)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.bn1(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.bn2(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.bn3(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.bn2(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

#### Function to Compute Score of the Model

In [ ]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

In [ ]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

In [ ]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [ ]:
def train_evaluate():

    model = GraphSAGE3(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GraphSAGE3(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.480 | Valid Score: 0.815
 
Epoch: 1/100 | Best Valid Score Until Now: 0.815 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.372 | Valid Score: 0.824
 
Epoch: 2/100 | Best Valid Score Until Now: 0.824 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.338 | Valid Score: 0.854
 
Epoch: 3/100 | Best Valid Score Until Now: 0.854 

Patience 1
Epoch: 4/100 | Training Loss: 0.312 | Valid Score: 0.841
 
Epoch: 4/100 | Best Valid Score Until Now: 0.854 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.291 | Valid Score: 0.855
 
Epoch: 5/100 | Best Valid Score Until Now: 0.855 

Patience 1
Epoch: 6/100 | Training Loss: 0.283 | Valid Score: 0.841
 
Epoch: 6/100 | Best Valid Score Until Now: 0.855 

Patience 2
Epoch: 7/100 | Training Loss: 0.254 | Valid Score: 0.846
 
Epoch: 7/100 | Best Valid Score Until Now: 0.855 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.238 | Valid Score: 0.864
 
Epoch: 8/100 | Best Valid Score Until Now: 0.864 

Patienc

# GAT 1

In [ ]:
from dgl.nn.pytorch import GATConv
class GATConv1(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        # Number of attention heads
        self.num_heads = self.config.get('num_heads', 1)

        # Dropout probability
        self.dropout = self.config.get('dropout', 0.0)

        # GAT layer
        self.conv1 = GATConv(
            self.node_feature_size,
            self.hidden_size,
            num_heads=self.num_heads,
            feat_drop=self.dropout,
            attn_drop=self.dropout,allow_zero_in_degree=True
        )

        # Linear layer
        self.fc = nn.Linear(
            self.hidden_size * self.num_heads,
            self.hidden_size
        )

        # GAT layer
        self.conv2 = GATConv(
            self.hidden_size,
            self.num_tasks,
            num_heads=1,
            feat_drop=self.dropout,
            attn_drop=self.dropout,allow_zero_in_degree=True
        )

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        # First GAT layer
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]).flatten(1)
        h = F.relu(h)
        h = self.fc(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Second GAT layer
        h = self.conv2(mol_dgl_graph, h).squeeze(1)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

#### Function to Compute Score of the Model

In [ ]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

In [ ]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

In [ ]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [ ]:
def train_evaluate():

    model = GATConv1(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GATConv1(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.561 | Valid Score: 0.772
 
Epoch: 1/100 | Best Valid Score Until Now: 0.772 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.466 | Valid Score: 0.817
 
Epoch: 2/100 | Best Valid Score Until Now: 0.817 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.425 | Valid Score: 0.820
 
Epoch: 3/100 | Best Valid Score Until Now: 0.820 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.423 | Valid Score: 0.838
 
Epoch: 4/100 | Best Valid Score Until Now: 0.838 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.402 | Valid Score: 0.838
 
Epoch: 5/100 | Best Valid Score Until Now: 0.838 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.404 | Valid Score: 0.844
 
Epoch: 6/100 | Best Valid Score Until Now: 0.844 

Patience 1
Epoch: 7/100 | Training Loss: 0.396 | Valid Score: 0.837
 
Epoch: 7/100 | Best Valid Score Until Now: 0.844 

Patience 2
Epoch: 8/100 | Training Loss: 0.395 | Valid Score: 0.813
 
Epoch: 8/100 | Best Valid Score Until Now: 0.844 

Pa

# GAT 2

In [ ]:
from dgl.nn.pytorch import GATConv
class GATConv2(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        # Number of attention heads
        self.num_heads = self.config.get('num_heads', 1)

        # Dropout probability
        self.dropout = self.config.get('dropout', 0.0)

        # GAT layer
        self.conv1 = GATConv(self.node_feature_size,self.hidden_size,num_heads=self.num_heads,feat_drop=self.dropout,attn_drop=self.dropout,allow_zero_in_degree=True)

        # Linear layer
        self.fc = nn.Linear(self.hidden_size * self.num_heads,self.hidden_size)

        # GAT layer
        self.conv2 = GATConv(self.hidden_size,self.hidden_size,num_heads=1,feat_drop=self.dropout,attn_drop=self.dropout,allow_zero_in_degree=True)

        # GAT layer
        self.conv3 = GATConv(self.hidden_size,self.num_tasks,num_heads=1,feat_drop=self.dropout,attn_drop=self.dropout,allow_zero_in_degree=True)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        # First GAT layer
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]).flatten(1)
        h = F.relu(h)
        h = self.fc(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Second GAT layer
        h = self.conv2(mol_dgl_graph, h).squeeze(1)
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Third GAT layer
        if self.num_tasks == 1:
            h = self.conv3(mol_dgl_graph, h).squeeze(1)
        else:
            hs = []
            for i in range(self.num_tasks):
                hi = self.conv3(mol_dgl_graph, h).squeeze(1)
                hs.append(hi)
            h = torch.stack(hs, dim=1)

        mol_dgl_graph.ndata["h"] = h

        if self.num_tasks == 1:
            return dgl.mean_nodes(mol_dgl_graph, "h")
        else:
            return dgl.mean_nodes(mol_dgl_graph, "h"), h

#### Function to Compute Score of the Model

In [ ]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

In [ ]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

In [ ]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [ ]:
def train_evaluate():

    model = GATConv2(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GATConv2(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.518 | Valid Score: 0.767
 
Epoch: 1/100 | Best Valid Score Until Now: 0.767 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.461 | Valid Score: 0.792
 
Epoch: 2/100 | Best Valid Score Until Now: 0.792 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.438 | Valid Score: 0.801
 
Epoch: 3/100 | Best Valid Score Until Now: 0.801 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.450 | Valid Score: 0.804
 
Epoch: 4/100 | Best Valid Score Until Now: 0.804 

Patience 1
Epoch: 5/100 | Training Loss: 0.428 | Valid Score: 0.798
 
Epoch: 5/100 | Best Valid Score Until Now: 0.804 

Patience 2
Epoch: 6/100 | Training Loss: 0.421 | Valid Score: 0.801
 
Epoch: 6/100 | Best Valid Score Until Now: 0.804 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.412 | Valid Score: 0.811
 
Epoch: 7/100 | Best Valid Score Until Now: 0.811 

Patience 1
Epoch: 8/100 | Training Loss: 0.410 | Valid Score: 0.799
 
Epoch: 8/100 | Best Valid Score Until Now: 0.811 

Patienc

#GAT 3


In [ ]:
from dgl.nn.pytorch import GATConv
from torch.nn import BatchNorm1d

class GATConv3(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        # Number of attention heads
        self.num_heads = self.config.get('num_heads', 1)

        # Dropout probability
        self.dropout = self.config.get('dropout', 0.0)

        # GAT layer
        self.conv1 = GATConv(self.node_feature_size, 64, num_heads=self.num_heads,
                             feat_drop=self.dropout, attn_drop=self.dropout,
                             allow_zero_in_degree=True)
        self.conv2 = GATConv(64, 128, num_heads=1, feat_drop=self.dropout,
                             attn_drop=self.dropout, allow_zero_in_degree=True)
        self.conv3 = GATConv(128, 256, num_heads=1, feat_drop=self.dropout,
                             attn_drop=self.dropout, allow_zero_in_degree=True)
        self.conv4 = GATConv(256, 128, num_heads=1, feat_drop=self.dropout,
                             attn_drop=self.dropout, allow_zero_in_degree=True)
        self.conv5 = GATConv(128, self.num_tasks, num_heads=1, feat_drop=self.dropout,
                             attn_drop=self.dropout, allow_zero_in_degree=True)

        # Linear layer
        self.fc1 = nn.Linear(64 * self.num_heads, 64)
        self.bn1 = BatchNorm1d(64)
        self.fc2 = nn.Linear(128 * self.num_heads, 128)
        self.bn2 = BatchNorm1d(128)
        self.fc3 = nn.Linear(256 * self.num_heads, 256)
        self.bn3 = BatchNorm1d(256)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        # First GAT layer
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]).flatten(1)
        h = F.relu(h)
        h = self.fc1(h)
        h = self.bn1(h)  # add batch normalization
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Second GAT layer
        h = self.conv2(mol_dgl_graph, h).squeeze(1)
        h = F.relu(h)
        h = self.fc2(h)
        h = self.bn2(h)  # add batch normalization
        h = F.dropout(h, p=self.dropout, training=self.training)

        h = self.conv3(mol_dgl_graph, h).squeeze(1)
        h = F.relu(h)
        h = self.fc3(h)
        h = self.bn3(h)  # add batch normalization
        h = F.dropout(h, p=self.dropout, training=self.training)

        h = self.conv4(mol_dgl_graph, h).squeeze(1)
        h = F.relu(h)
        h = self.fc2(h)
        h = self.bn2(h)  # add batch normalization
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Third GAT layer
        if self.num_tasks == 1:
            h = self.conv5(mol_dgl_graph, h).squeeze(1)
        else:
            hs = []
            for i in range(self.num_tasks):
                hi = self.conv3(mol_dgl_graph, h).squeeze(1)
                hs.append(hi)
            h = torch.stack(hs, dim=1)

        mol_dgl_graph.ndata["h"]= h

        if self.num_tasks == 1:
            return dgl.mean_nodes(mol_dgl_graph, "h")
        else:
            return dgl.mean_nodes(mol_dgl_graph, "h"), h

#### Function to Compute Score of the Model

In [ ]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

In [ ]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

In [ ]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [ ]:
def train_evaluate():

    model = GATConv3(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GATConv3(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.565 | Valid Score: 0.788
 
Epoch: 1/100 | Best Valid Score Until Now: 0.788 

Patience 1
Epoch: 2/100 | Training Loss: 0.437 | Valid Score: 0.779
 
Epoch: 2/100 | Best Valid Score Until Now: 0.788 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.402 | Valid Score: 0.809
 
Epoch: 3/100 | Best Valid Score Until Now: 0.809 

Patience 1
Epoch: 4/100 | Training Loss: 0.397 | Valid Score: 0.791
 
Epoch: 4/100 | Best Valid Score Until Now: 0.809 

Patience 2
Epoch: 5/100 | Training Loss: 0.379 | Valid Score: 0.796
 
Epoch: 5/100 | Best Valid Score Until Now: 0.809 

Patience 3
Epoch: 6/100 | Training Loss: 0.387 | Valid Score: 0.800
 
Epoch: 6/100 | Best Valid Score Until Now: 0.809 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.372 | Valid Score: 0.817
 
Epoch: 7/100 | Best Valid Score Until Now: 0.817 

Patience 1
Epoch: 8/100 | Training Loss: 0.372 | Valid Score: 0.804
 
Epoch: 8/100 | Best Valid Score Until Now: 0.817 

Patience 2
Epoch: